# Stable Diffusion v2.1 using OpenVINO TorchDynamo backend

Stable Diffusion v2 is the next generation of Stable Diffusion model a Text-to-Image latent diffusion model created by the researchers and engineers from [Stability AI](https://stability.ai/) and [LAION](https://laion.ai/). 

General diffusion models are machine learning systems that are trained to denoise random gaussian noise step by step, to get to a sample of interest, such as an image.
Diffusion models have shown to achieve state-of-the-art results for generating image data. But one downside of diffusion models is that the reverse denoising process is slow. In addition, these models consume a lot of memory because they operate in pixel space, which becomes unreasonably expensive when generating high-resolution images. Therefore, it is challenging to train these models and also use them for inference. OpenVINO brings capabilities to run model inference on Intel hardware and opens the door to the fantastic world of diffusion models for everyone!

This notebook demonstrates how to run stable diffusion model using [Diffusers](https://huggingface.co/docs/diffusers/index) library and [OpenVINO `TorchDynamo` backend](https://docs.openvino.ai/2024/openvino-workflow/torch-compile.html) for Text-to-Image and Image-to-Image generation tasks.

Notebook contains the following steps:

1. Create pipeline with PyTorch models.
2. Add OpenVINO optimization using OpenVINO TorchDynamo backend.
3. Run Stable Diffusion pipeline with OpenVINO.




#### Table of contents:

- [Prerequisites](#Prerequisites)
- [Stable Diffusion with Diffusers library](#Stable-Diffusion-with-Diffusers-library)
- [OpenVINO TorchDynamo backend](#OpenVINO-TorchDynamo-backend)
    - [Run Image generation](#Run-Image-generation)
- [Interactive demo](#Interactive-demo)
- [Support for Automatic1111 Stable Diffusion WebUI](#Support-for-Automatic1111-Stable-Diffusion-WebUI)



## Prerequisites
[back to top ⬆️](#Table-of-contents:)

In [1]:
%pip install -q "torch>=2.2" transformers diffusers "gradio>=4.19" ipywidgets --extra-index-url https://download.pytorch.org/whl/cpu
%pip install -q "openvino>=2024.1.0"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import gradio as gr
import random
import torch
import time

from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
import ipywidgets as widgets

/home/dlyakhov/env/tmp/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/dlyakhov/env/tmp/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/dlyakhov/env/tmp/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/dlyakhov/env/tmp/lib/python3.10/site-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOu

## Stable Diffusion with Diffusers library
[back to top ⬆️](#Table-of-contents:)

To work with Stable Diffusion v2.1, we will use Hugging Face Diffusers library. To experiment with Stable Diffusion models, Diffusers exposes the [StableDiffusionPipeline](https://huggingface.co/docs/diffusers/using-diffusers/conditional_image_generation) and [StableDiffusionImg2ImgPipeline](https://huggingface.co/docs/diffusers/using-diffusers/img2img) similar to the other [Diffusers pipelines](https://huggingface.co/docs/diffusers/api/pipelines/overview). The code below demonstrates how to create the `StableDiffusionPipeline` using `stable-diffusion-2-1-base` model:

In [3]:
model_id = "stabilityai/stable-diffusion-2-1-base"

# Pipeline for text-to-image generation
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

## OpenVINO TorchDynamo backend
[back to top ⬆️](#Table-of-contents:)

The [OpenVINO TorchDynamo backend](https://docs.openvino.ai/2024/openvino-workflow/torch-compile.html) lets you enable [OpenVINO](https://docs.openvino.ai/2024/home.html) support for PyTorch models with minimal changes to the original PyTorch script. It speeds up PyTorch code by JIT-compiling it into optimized kernels. By default, Torch code runs in eager-mode, but with the use of torch.compile it goes through the following steps:
1. Graph acquisition - the model is rewritten as blocks of subgraphs that are either:
   - compiled by TorchDynamo and “flattened”,
   - falling back to the eager-mode, due to unsupported Python constructs (like control-flow code).
2. Graph lowering - all PyTorch operations are decomposed into their constituent kernels specific to the chosen backend.
3. Graph compilation - the kernels call their corresponding low-level device-specific operations.

Select device for inference and enable or disable saving the optimized model files to a hard drive, after the first application run. This makes them available for the following application executions, reducing the first-inference latency. Read more about available [Environment Variables options](https://docs.openvino.ai/2024/openvino-workflow/torch-compile.html#options)

In [4]:
import openvino as ov

core = ov.Core()
device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value="CPU",
    description="Device:",
    disabled=False,
)

device

Dropdown(description='Device:', options=('CPU', 'GPU.0', 'GPU.1', 'GPU.2', 'AUTO'), value='CPU')

In [5]:
model_caching = widgets.Dropdown(
    options=[True, False],
    value=True,
    description="Model caching:",
    disabled=False,
)

model_caching

Dropdown(description='Model caching:', options=(True, False), value=True)

To use [`torch.compile()` method](https://pytorch.org/tutorials/intermediate/torch_compile_tutorial.html), you just need to add an import statement and define the OpenVINO backend:

In [10]:
import torch
from torch._export import capture_pre_autograd_graph
import torch.ao.quantization.quantizer.x86_inductor_quantizer as xiq
from torch.ao.quantization.quantizer.x86_inductor_quantizer import X86InductorQuantizer
from torch.ao.quantization.quantize_pt2e import convert_pt2e
from torch.ao.quantization.quantize_pt2e import prepare_pt2e
from torch.export import Dim
from itertools import islice

constraint = Dim("ids_att_mask_const", max=76)
dynamic_shapes = {"input_ids": {1: constraint}, "attention_mask": {1: constraint},
                  "pixel_values": {}}

def quantize(model, calibration_dataset, subset_size: int = 300):
    example_args = next(iter(calibration_dataset))
    with torch.no_grad():
        # Static
        exported_model = capture_pre_autograd_graph(model, args=example_args, kwargs={})
        # Dynamic
        #exported_model = capture_pre_autograd_graph(model, args=(), kwargs=example_kwargs,
        #                                            dynamic_shapes=dynamic_shapes)
        # torch 2.3.1
        #exported_model = capture_pre_autograd_graph(model, args=(), kwargs=example_kwargs, constraints=[
        #    dynamic_dim(example_kwargs["input_ids"], 1) == dynamic_dim(example_kwargs["attention_mask"], 1),
        #    # Torch.fx said I need this  constraint as well 0-0
        #    dynamic_dim(example_kwargs["input_ids"], 1) < 77,
        #])

    open("c.py", "w").write(exported_model.code)
    quantizer = X86InductorQuantizer()
    quantizer.set_global(xiq.get_default_x86_inductor_quantization_config())

    prepared_model = prepare_pt2e(exported_model, quantizer)
    from tqdm import tqdm

    for inp  in islice(tqdm(calibration_dataset), subset_size):
        # Static
        # prepared_model(**example_kwargs)
        # Dynamic
        prepared_model(*inp)
    converted_model = convert_pt2e(prepared_model)
    return converted_model

In [11]:
example_input = [(torch.ones(2, 4, 64, 64), 981, torch.ones(2, 77, 1024))]
unet_quantized = quantize(pipe.unet, example_input)

100%|██████████| 1/1 [00:02<00:00,  2.09s/it]


In [12]:
unet_quantized(*example_input[0])

UNet2DConditionOutput(sample=tensor([[[[0.2550, 0.3102, 0.2675,  ..., 0.2527, 0.3319, 0.4627],
          [0.1833, 0.2141, 0.1447,  ..., 0.1216, 0.2257, 0.4241],
          [0.1452, 0.1510, 0.0921,  ..., 0.0664, 0.1770, 0.3639],
          ...,
          [0.1341, 0.1622, 0.1039,  ..., 0.0903, 0.2078, 0.3962],
          [0.2218, 0.2442, 0.1896,  ..., 0.1797, 0.2569, 0.4396],
          [0.3356, 0.2939, 0.2538,  ..., 0.2442, 0.3156, 0.4656]],

         [[0.3784, 0.1833, 0.1533,  ..., 0.1446, 0.1966, 0.0815],
          [0.3786, 0.1726, 0.1422,  ..., 0.1422, 0.1735, 0.0477],
          [0.3865, 0.1689, 0.1423,  ..., 0.1444, 0.1889, 0.0547],
          ...,
          [0.3688, 0.1519, 0.1203,  ..., 0.1325, 0.1775, 0.0477],
          [0.3627, 0.1491, 0.1245,  ..., 0.1278, 0.1777, 0.0442],
          [0.4313, 0.2987, 0.2745,  ..., 0.2742, 0.3371, 0.1511]],

         [[0.6024, 0.4573, 0.4148,  ..., 0.4265, 0.4101, 0.5287],
          [0.3853, 0.1774, 0.1664,  ..., 0.1485, 0.1382, 0.3070],
          [0.

In [6]:
# this import is required to activate the openvino backend for torchdynamo
import openvino.torch  # noqa: F401

pipe.unet = torch.compile(
    pipe.unet,
    backend="openvino",
    options={"device": device.value, "model_caching": model_caching.value},
)

> **Note**: Read more about available [OpenVINO backends](https://docs.openvino.ai/2024/openvino-workflow/torch-compile.html#how-to-use)

### Run Image generation
[back to top ⬆️](#Table-of-contents:)


In [12]:
prompt = "a photo of an astronaut riding a horse on mars"
image = pipe(prompt).images[0]
image

/home/dlyakhov/env/tmp/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:222: FutureWarning: Accessing config attribute `__iter__` directly via 'UNet2DConditionModel' object attribute is deprecated. Please access '__iter__' over 'UNet2DConditionModel's config object instead, e.g. 'unet.config.__iter__'.
  return getattr(self._orig_mod, name)
OpenVINO execution failed. Falling back to native PyTorch execution.


  0%|          | 0/50 [00:00<?, ?it/s]

# Interactive demo
[back to top ⬆️](#Table-of-contents:)

Now you can start the demo, choose the inference mode, define prompts (and input image for Image-to-Image generation) and run inference pipeline.
Optionally, you can also change some input parameters.

In [9]:
time_stamps = []


def callback(iter, t, latents):
    time_stamps.append(time.time())


def error_str(error, title="Error"):
    return (
        f"""#### {title}
            {error}"""
        if error
        else ""
    )


def on_mode_change(mode):
    return gr.update(visible=mode == modes["img2img"]), gr.update(visible=mode == modes["txt2img"])


def inference(
    inf_mode,
    prompt,
    guidance=7.5,
    steps=25,
    width=768,
    height=768,
    seed=-1,
    img=None,
    strength=0.5,
    neg_prompt="",
):
    if seed == -1:
        seed = random.randint(0, 10000000)
    generator = torch.Generator().manual_seed(seed)
    res = None

    global time_stamps, pipe
    time_stamps = []
    try:
        if inf_mode == modes["txt2img"]:
            if type(pipe).__name__ != "StableDiffusionPipeline":
                pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
                pipe.unet = torch.compile(pipe.unet, backend="openvino")
            res = pipe(
                prompt,
                negative_prompt=neg_prompt,
                num_inference_steps=int(steps),
                guidance_scale=guidance,
                width=width,
                height=height,
                generator=generator,
                callback=callback,
                callback_steps=1,
            ).images
        elif inf_mode == modes["img2img"]:
            if img is None:
                return (
                    None,
                    None,
                    gr.update(
                        visible=True,
                        value=error_str("Image is required for Image to Image mode"),
                    ),
                )
            if type(pipe).__name__ != "StableDiffusionImg2ImgPipeline":
                pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
                pipe.unet = torch.compile(pipe.unet, backend="openvino")
            res = pipe(
                prompt,
                negative_prompt=neg_prompt,
                image=img,
                num_inference_steps=int(steps),
                strength=strength,
                guidance_scale=guidance,
                generator=generator,
                callback=callback,
                callback_steps=1,
            ).images
    except Exception as e:
        return None, None, gr.update(visible=True, value=error_str(e))

    warmup_duration = time_stamps[1] - time_stamps[0]
    generation_rate = (steps - 1) / (time_stamps[-1] - time_stamps[1])
    res_info = "Warm up time: " + str(round(warmup_duration, 2)) + " secs "
    if generation_rate >= 1.0:
        res_info = res_info + ", Performance: " + str(round(generation_rate, 2)) + " it/s "
    else:
        res_info = res_info + ", Performance: " + str(round(1 / generation_rate, 2)) + " s/it "

    return (
        res,
        gr.update(visible=True, value=res_info),
        gr.update(visible=False, value=None),
    )


modes = {
    "txt2img": "Text to Image",
    "img2img": "Image to Image",
}

with gr.Blocks(css="style.css") as demo:
    gr.HTML(
        f"""
            Model used: {model_id}         
        """
    )
    with gr.Row():
        with gr.Column(scale=60):
            with gr.Group():
                prompt = gr.Textbox(
                    "a photograph of an astronaut riding a horse",
                    label="Prompt",
                    max_lines=2,
                )
                neg_prompt = gr.Textbox(
                    "frames, borderline, text, character, duplicate, error, out of frame, watermark, low quality, ugly, deformed, blur",
                    label="Negative prompt",
                )
                res_img = gr.Gallery(label="Generated images", show_label=False)
            error_output = gr.Markdown(visible=False)

        with gr.Column(scale=40):
            generate = gr.Button(value="Generate")

            with gr.Group():
                inf_mode = gr.Dropdown(list(modes.values()), label="Inference Mode", value=modes["txt2img"])

                with gr.Column(visible=False) as i2i:
                    image = gr.Image(label="Image", height=128, type="pil")
                    strength = gr.Slider(
                        label="Transformation strength",
                        minimum=0,
                        maximum=1,
                        step=0.01,
                        value=0.5,
                    )

            with gr.Group():
                with gr.Row() as txt2i:
                    width = gr.Slider(label="Width", value=512, minimum=64, maximum=1024, step=8)
                    height = gr.Slider(label="Height", value=512, minimum=64, maximum=1024, step=8)

            with gr.Group():
                with gr.Row():
                    steps = gr.Slider(label="Steps", value=20, minimum=1, maximum=50, step=1)
                    guidance = gr.Slider(label="Guidance scale", value=7.5, maximum=15)

                seed = gr.Slider(-1, 10000000, label="Seed (-1 = random)", value=-1, step=1)

            res_info = gr.Markdown(visible=False)

    inf_mode.change(on_mode_change, inputs=[inf_mode], outputs=[i2i, txt2i], queue=False)

    inputs = [
        inf_mode,
        prompt,
        guidance,
        steps,
        width,
        height,
        seed,
        image,
        strength,
        neg_prompt,
    ]

    outputs = [res_img, res_info, error_output]
    prompt.submit(inference, inputs=inputs, outputs=outputs)
    generate.click(inference, inputs=inputs, outputs=outputs)

try:
    demo.queue().launch(debug=True)
except Exception:
    demo.queue().launch(share=True, debug=True)

# if you are launching remotely, specify server_name and server_port
# demo.launch(server_name='your server name', server_port='server port in int')
# Read more in the docs: https://gradio.app/docs/

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


## Support for Automatic1111 Stable Diffusion WebUI
[back to top ⬆️](#Table-of-contents:)

Automatic1111 Stable Diffusion WebUI is an open-source repository that hosts a browser-based interface for the Stable Diffusion based image generation. It allows users to create realistic and creative images from text prompts. Stable Diffusion WebUI is supported on Intel CPUs, Intel integrated GPUs, and Intel discrete GPUs by leveraging OpenVINO torch.compile capability. Detailed instructions are available in[ Stable Diffusion WebUI repository](https://github.com/openvinotoolkit/stable-diffusion-webui/wiki/Installation-on-Intel-Silicon).